In [1]:
library(tidyverse)
library(ggplot2)
library(patchwork)
source("helper.R")

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.1.2     v dplyr   1.0.6
v tidyr   1.1.3     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
sinfo_path = "../../output/sims_info.tsv"
sinfoh_path = "../../output/header_sims_info.tsv"

In [3]:
win_size = 1000000
win_len = -1
padding = 0
rescf = 1
prop_acc=0.4

In [4]:
fig_prefix = "cor-pidxy-dT"

In [5]:
sims_path = "../../output/sims_results/"
varmut_sims_path = "../../output/varmut/sims_results/"
data_path = paste0("../../output/greatapes-diversity-data/",fig_prefix,"_win-size_",as.integer(win_size),"_merged-mask_True_prop-acc_",prop_acc, ".rds")

In [6]:
# loading sims info table
sinfo = read.table(sinfo_path, sep="\t")
h = read.table(sinfoh_path, sep="\t", header=TRUE)
cnames = colnames(h)
if (length(sinfo) == length(h)+1) {
    cnames = c(cnames, "poscoefdecayeps")
}
colnames(sinfo) = cnames

In [7]:
#diff_cols = c("delrate","delcoef","posrate","poscoef","mu", "rescf", "win_len", "padding", "chr")
diff_cols = c("delrate","delcoef","posrate","poscoef","mu", "poscoefdecayeps", "rescf", "win_len", "padding", "chr")

In [8]:
wsize_str = paste0("win-size_",as.integer(win_size))

In [9]:
sims_folders = paste0(sims_path,list.files(sims_path)[grep(wsize_str, list.files(sims_path))])

In [10]:
varmut_sims_folders = paste0(varmut_sims_path,list.files(varmut_sims_path)[grep(wsize_str, list.files(varmut_sims_path))])

In [11]:
sims_folders = c(sims_folders, varmut_sims_folders)

In [12]:
simresults = as.data.frame(do.call(rbind, lapply(X = sims_folders, function(x) unlist(meta_from_fname(x,prop=prop_acc,suffix="$")))), stringsAsFactors =FALSE)

[1] "entrei"
[1] "entrei"
[1] "entrei"
[1] "entrei"
[1] "entrei"
[1] "entrei"
[1] "entrei"
[1] "entrei"
[1] "entrei"
[1] "entrei"


In [13]:
#sims_folders

In [14]:
simresults$folderpath = sims_folders

In [15]:
tmp = unique(sinfo[sinfo$sup_rand_id %in% simresults$sup_rand_id,c(diff_cols,"sup_rand_id")])

In [16]:
sinfo[sinfo$sup_rand_id=="O814WK8MN3UOF27JIR",]["rand_id"]

,rand_id
,<fct>
131,6DRX6RL6BNZCHP8
2558,6DRX6RL6BNZCHP8
3323,6DRX6RL6BNZCHP8
5056,6DRX6RL6BNZCHP8
5117,6DRX6RL6BNZCHP8
5649,6DRX6RL6BNZCHP8
5812,6DRX6RL6BNZCHP8
6261,6DRX6RL6BNZCHP8
7464,6DRX6RL6BNZCHP8


In [17]:
simresults = left_join(simresults, tmp)

Joining, by = "sup_rand_id"



In [18]:
simresults = simresults[simresults$win_len==win_len & simresults$padding == padding & simresults$rescf==1 & as.numeric(simresults$win_size) == win_size,]

In [19]:
win_size

[1] 1e+06

In [20]:
simresults[is.na(simresults$poscoefdecayeps),]$poscoefdecayeps = 0

In [21]:
simresults$selregime = ifelse(simresults$posrate<1e-16,ifelse(simresults$delrate<1e-16, ifelse(simresults$sigma==0, "neutral", "varmut"), "del_only"), ifelse(simresults$delrate<1e-16, "pos_only", "both"))

In [22]:
simresults$seldesc = apply(simresults,1, function(x) paste("mu-D=", x["delrate"], " mu-B=", x["posrate"], ifelse(x['selregime'] %in% c("del_only", "both"), paste0("\ncoef-D=", x["delcoef"]), ""), ifelse(x['selregime'] %in% c("both", "pos_only"), paste0("\ncoef-B=", x["poscoef"], " decay-B=", x["poscoefdecayeps"]), ""), ifelse(x["selregime"]=="varmut", paste0(" mut-sd=", x['sigma']),""), sep=""))

In [23]:
#simresults

In [24]:
simresults$sigma = as.numeric(simresults$sigma)

In [25]:
tmp = simresults
ordered = c("neutral", "del_only", "pos_only", "both", "varmut")
classes = unique(tmp$selregime)
tmp$selregime = factor(tmp$selregime, levels=classes[order(match(classes,ordered))])
tmp = tmp[
  with(tmp, order(selregime,posrate,-poscoefdecayeps,delrate,poscoef,delcoef,sigma)),
]

In [26]:
order(tmp$selregime)

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
[26] 26 27 28 29 30 31 32 33 34 35

In [27]:
rownames(tmp) <- NULL
plots = apply(tmp,1,
              function(x) readRDS(paste0(x["folderpath"],
                                         "/",fig_prefix, "_",x["desc"],".rds"))+ggtitle(x["seldesc"]))
#plots = c(readRDS(data_path)+ggtitle("data"), plots)

In [28]:
plots=c(list(readRDS(data_path)+ggtitle("data")), plots)

In [29]:
tab=table(tmp$selregime)

In [30]:
tab = c("data"=1, tab)

In [31]:
tab

data  neutral del_only pos_only     both   varmut 
       1        1        6        2       16       10

In [32]:
areas = list()
row = 1
for (regime in names(tab)) {
    col = 1
    for(i in 1:tab[regime]) {
        areas = c(areas, list(area(row,col)))
        col = col + 1
    }
    row = row + 1
}

In [33]:
areas = (do.call(c, areas))

In [34]:
#design=""
#for (regime in names(tab)) {
#    if (design != "") design = paste0(design,"\n")
#    num_cols = 1:tab[regime]
#    num_blanks = max(tab)-tab[regime]
#    if (regime == "data") {
#        indexes = c(1)
#    } else {
#        indexes = 1+as.numeric(rownames(tmp[tmp$selregime==regime,]))        
#    }
#    print(LETTERS[indexes])
#    print(rep("#", num_blanks))
#    add = paste0(paste0(LETTERS[indexes],collapse=""), paste0(rep("#", num_blanks),collapse=""), collapse="")
#    print(design)
#    print(add)
#    design = paste0(design,add)
#    print(design)
#}

In [35]:
tmp$selregime

[1] neutral  del_only del_only del_only del_only del_only del_only pos_only
 [9] pos_only both     both     both     both     both     both     both    
[17] both     both     both     both     both     both     both     both    
[25] both     varmut   varmut   varmut   varmut   varmut   varmut   varmut  
[33] varmut   varmut   varmut  
Levels: neutral del_only pos_only both varmut

In [36]:
panel_plot = wrap_plots(plots, nrow=length(classes), design=areas, guides="collect")

In [37]:
wstr = paste0('win-size_',as.integer(win_size))
plots_folder_path = paste0("../../output/data_and_sims/", wstr,"/", fig_prefix)

dname = paste0(plots_folder_path)
if(! dir.exists(dname))
        dir.create(dname, recursive=TRUE)


In [39]:
ggsave(filename=paste0(plots_folder_path,"/","panel_sims_and_data_win-size_.pdf"), plot=panel_plot, width=100, height=25, scale=1.75, units="cm", limitsize=FALSE)

Warning message:
"Removed 283 rows containing missing values (geom_point)."
Warning message:
"Removed 294 rows containing missing values (geom_point)."
Warning message:
"Removed 159 rows containing missing values (geom_point)."
Warning message:
"Removed 88 rows containing missing values (geom_point)."
Warning message:
"Removed 80 rows containing missing values (geom_point)."
Warning message:
"Removed 1 rows containing missing values (geom_point)."
Warning message:
"Removed 148 rows containing missing values (geom_point)."
Warning message:
"Removed 1 rows containing missing values (geom_point)."
Warning message:
"Removed 110 rows containing missing values (geom_point)."
Warning message:
"Removed 127 rows containing missing values (geom_point)."
Warning message:
"Removed 39 rows containing missing values (geom_point)."
Warning message:
"Removed 113 rows containing missing values (geom_point)."
Warning message:
"Removed 118 rows containing missing values (geom_point)."
Warning message:
"R

In [ ]:
tmp

In [40]:
fig_prefix = "subset-chr12-landscapes"
plots_folder_path = paste0("../../output/data_and_sims/", wstr,"/", fig_prefix)

dname = paste0(plots_folder_path)
if(! dir.exists(dname))
        dir.create(dname, recursive=TRUE)

In [41]:
data_path2 = paste0("../../output/greatapes-diversity-data/figs/",fig_prefix,"_win-size_",as.integer(win_size),"_merged-mask_True_prop-acc_",prop_acc, ".pdf")

In [42]:
plots_paths = apply(tmp,1,
              function(x) paste0(x["folderpath"],
                                         "/figs/",fig_prefix,"_",x["desc"],".pdf"))
tmp$newfname = gsub(" ", "_", tmp[,"seldesc"])
tmp$newfname = gsub("\n", "_", tmp[,"newfname"])

new_paths = apply(tmp,1,
                 function(x) paste0(plots_folder_path,"/",x["selregime"],"_", x['newfname'],".pdf"))

In [43]:
plots_folder_path

[1] "../../output/data_and_sims/win-size_1000000/subset-chr12-landscapes"

In [44]:
file.copy(plots_paths, new_paths)
file.copy(data_path2, paste0(plots_folder_path,"/data.pdf"))

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[31] TRUE TRUE TRUE TRUE TRUE

[1] TRUE

In [45]:
plots_paths

[1] "../../output/sims_results/sup-rand-id_8MF0I020MWGI6N41PH_rep_0_win-size_1000000_sample-size_10/figs/subset-chr12-landscapes_sup-rand-id_8MF0I020MWGI6N41PH_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4.pdf"                                                                                                              
 [2] "../../output/sims_results/sup-rand-id_RNX4L54J663QN2SY5X_rep_0_win-size_1000000_sample-size_10/figs/subset-chr12-landscapes_sup-rand-id_RNX4L54J663QN2SY5X_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4.pdf"                                                                                                              
 [3] "../../output/sims_results/sup-rand-id_SIN0QA3J1CBIYBO7D7_rep_0_win-size_1000000_sample-size_10/figs/subset-chr12-landscapes_sup-rand-id_SIN0QA3J1CBIYBO7D7_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4.pdf"                                                                                                              
 [4] "../../output/sims_results/sup-rand-id_UTJG9R531BVSR6UA41_rep_0_win-size_1000000_sample-size_10/figs/subset-chr12-landscapes_sup-rand-id_UTJG9R531BVSR6UA41_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4.pdf"                                                                                                              
 [5] "../../output/sims_results/sup-rand-id_O814WK8MN3UOF27JIR_rep_0_win-size_1000000_sample-size_10/figs/subset-chr12-landscapes_sup-rand-id_O814WK8MN3UOF27JIR_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4.pdf"                                                                                                              
 [6] "../../output/sims_results/sup-rand-id_AA9GUSRHWOA341WGC4_rep_0_win-size_1000000_sample-size_10/figs/subset-chr12-landscapes_sup-rand-id_AA9GUSRHWOA341WGC4_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4.pdf"                                                                                                              
 [7] "../../output/sims_results/sup-rand-id_CR1UIIF8NRSPZ94LGF_rep_0_win-size_1000000_sample-size_10/figs/subset-chr12-landscapes_sup-rand-id_CR1UIIF8NRSPZ94LGF_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4.pdf"                                                                                                              
 [8] "../../output/sims_results/sup-rand-id_4GROMS6FHMNW5X9L72_rep_0_win-size_1000000_sample-size_10/figs/subset-chr12-landscapes_sup-rand-id_4GROMS6FHMNW5X9L72_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4.pdf"                                                                                                              
 [9] "../../output/sims_results/sup-rand-id_HC27BSIEQBYZJHHUQV_rep_0_win-size_1000000_sample-size_10/figs/subset-chr12-landscapes_sup-rand-id_HC27BSIEQBYZJHHUQV_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4.pdf"                                                                                                              
[10] "../../output/sims_results/sup-rand-id_QHYU2J74DGPZWTJKGK_rep_0_win-size_1000000_sample-size_10/figs/subset-chr12-landscapes_sup-rand-id_QHYU2J74DGPZWTJKGK_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4.pdf"                                                                                                              
[11] "../../output/sims_results/sup-rand-id_9WZ486SOY2Q604EOMM_rep_0_win-size_1000000_sample-size_10/figs/subset-chr12-landscapes_sup-rand-id_9WZ486SOY2Q604EOMM_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4.pdf"                                                                                                              
[12] "../../output/sims_results/sup-rand-id_HVD283AEPBL88J4NUN_rep_0_win-size_1000000_sample-size_10/figs/subset-chr12-landscapes_sup-rand-id_HVD283AEPBL88J4NUN_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4.pdf"                                                                                                              
[13] "../../output/sims_results/sup-rand-id_Z79XUJ3SLXIAOKPITD_rep_0_win-size

In [ ]:
tmp[1,"desc"]